# CONTROL DE OBRAS

La finalidad del presente reporte es cruzar la información de ejecución de obras entre Icaro y el Resumen de Rendiciones por Proveedor del SGF. En la medida que la información de Icaro haya sido previamente cruzada con SIIF, este control permitirá detectar, en mayor medida, desvíos en la carga del SIIF.

## Datos iniciales

El único dato que debería modificarse es el relativo al ejercicio sobre el cual se pretende trabajar.

In [2]:
from invicoctrlpy.gastos.control_obras.control_obras import ControlObras

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('mode.chained_assignment',None)

import itables
itables.options.lengthMenu = [10, 50, 100, -1]
itables.options.maxBytes = 0

from  pivottablejs import pivot_ui

EJERCICIO = '2022'
UPDATE_DB = True
control_obras = ControlObras(ejercicio=EJERCICIO, update_db=UPDATE_DB)

## Controles

### Control por Mes y Cuenta Corrientes

El presente reporte agrupa la información del SGF y el SSCC por mes y cta_cte para luego cruzarlos. En la última columna se expone las diferencias detectas. 

In [3]:
control_mes_cta_cte = control_obras.control_mes_cta_cte()
itables.show(control_mes_cta_cte, column_filters="footer", dom="lrtip")

## Tablas Auxiliares

### Icaro neto de Deuda Flotante

In [4]:
icaro = control_obras.icaro_neto_rdeu
itables.show(icaro, column_filters="footer", dom="lrtip")

### Deuda Flotante SIIF (rdeu012)

In [5]:
rdeu = control_obras.siif_rdeu012
itables.show(rdeu, column_filters="footer", dom="lrtip")

### Resumen de Rendiciones por Proveedor SGF (depurado)

In [6]:
sgf = control_obras.sgf_resumen_rend_cuit
itables.show(sgf, column_filters="footer", dom="lrtip")

## EXTRA:

### Excel

In [7]:
with pd.ExcelWriter('control_obras.xlsx') as writer:
    control_mes_cta_cte.to_excel(writer, sheet_name='control_obras', index=False)
    icaro.to_excel(writer, sheet_name='icaro', index=False)
    rdeu.to_excel(writer, sheet_name='rdeu', index=False)
    sgf.to_excel(writer, sheet_name='sgf', index=False)

### Tablas Dinámicas

Por separado, se incluye una tabla dinámica con toda la información de Icaro (neteado de rdeu) y del Resumen de Rendiciones por Proveedor (SGF depurado) para el ejercicio bajo análisis. Ambas bases de datos fueron pivoteadas en función de los campos mes, fecha, cta_cte y cuit. A su vez, se incluyó un campo adicional, sobre el cual debería realizarse los cálculos agregados, llamado **diferencia** el cual surge de resta entre la ejecución Icaro y el importe bruto del SGF.

In [8]:
pivot_ui(control_obras.control_completo(), 'tabla_dinamica_control_obras.html')